In [1]:
import cv2
import pandas as pd
import numpy as np
import os

# Parameters
input_video_path = "demo_signer.mp4"  # Change to your video file
input_parquet_path = "demo_user_demo_sign_after.parquet"  # Change to your .parquet file
output_video_path = "demo_signer_keypoints.mp4"

# Load landmarks
df = pd.read_parquet(input_parquet_path)

# Open video
cap = cv2.VideoCapture(input_video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # Get keypoints for this frame
    frame_landmarks = df[df['frame'] == frame_idx]
    for l_type, color in zip(['left_hand','right_hand','face','pose'], [(255,0,0),(0,0,255),(0,255,0),(255,165,0)]):
        pts = frame_landmarks[frame_landmarks['type'] == l_type]
        for _, row in pts.iterrows():
            # Convert normalized x,y to pixel coordinates
            x = int(row['x'] * frame_width)
            y = int(row['y'] * frame_height)
            cv2.circle(frame, (x, y), 3, color, -1)
    out.write(frame)
    frame_idx += 1
cap.release()
out.release()
print(f"Saved video with keypoints to {output_video_path}")


Saved video with keypoints to demo_signer_keypoints.mp4
